<div style="text-align: right;">
  <img src="https://raw.githubusercontent.com/exasol/ai-lab/refs/heads/main/assets/Exasol_Logo_2025_Dark.svg" style="width:200px; margin: 10px;" />
</div>

# Zero-shot classification model

In this notebook we will load and use a zero shot classification language model. Learn about the Zero Shot Classification task <a href="https://huggingface.co/tasks/zero-shot-classification" target="_blank" rel="noopener">here</a>. Please also refer to the Transformer Extension <a href="https://github.com/exasol/transformers-extension/blob/main/doc/user_guide/user_guide.md" target="_blank" rel="noopener">User Guide</a> to find more information about the UDF used in this notebook.

We will be running SQL queries using <a href="https://github.com/ploomber/jupysql" target="_blank" rel="noopener">JupySQL</a>SQL Magic.

## Prerequisites

Prior to using this notebook the following steps need to be completed:
1. [Configure the AI Lab](../main_config.ipynb).
2. [Initialize the Transformer Extension](te_init.ipynb).

## Setup

### Open Secure Configuration Storage

In [ ]:
%run ../utils/access_store_ui.ipynb
display(get_access_store_ui('../'))

Let's bring up JupySQL and connect to the database via SQLAlchemy. Please refer to the documentation of <a href="https://github.com/exasol/sqlalchemy-exasol" target="_blank" rel="noopener">sqlalchemy-exasol</a> for details on how to connect to the database using the Exasol SQLAlchemy driver.

In [ ]:
%run ../utils/jupysql_init.ipynb

## Get a language model

To demonstrate the zero shot classification task we will use the [Cross-Encoder for Natural Language Inference](https://huggingface.co/cross-encoder/nli-deberta-base) model.

We need to load the model from the Hugging Face Hub into the [BucketFS](https://docs.exasol.com/db/latest/database_concepts/bucketfs/bucketfs.htm). This could potentially be a long process, depending on the speed of the database connection. Unfortunately, we cannot tell exactly when it has finished. The notebook's hourglass may not be a reliable indicator. BucketFS will still be doing some work when the call issued by the notebook returns. Please wait for a few moments after that, before querying the model.

You might see a warning that some weights are newly initialized and the model should be trained on a down-stream task. Please ignore this warning. For the purpose of this demonstration, it is not important, the model should still be able to produce some meaningful output.

In [ ]:
from exasol.nb_connector.model_installation import install_model, TransformerModel
from transformers import AutoModelForSequenceClassification

# This is the name of the model at the Hugging Face Hub
MODEL_NAME = 'cross-encoder/nli-deberta-base'
install_model(ai_lab_config, TransformerModel(MODEL_NAME, 'zero_shot_classification', AutoModelForSequenceClassification))

## Use the language model

Below is a text that we will try to classify using labels that were not used during the model training. Out of five suggested labels the first two are much more relevant than the others. We expect the model to give them significantly higher score.

In [ ]:
# Text to be classified.
MY_TEXT = """
A new model offers an explanation for how the Galilean satellites formed around the solar system’s largest world. 
Konstantin Batygin did not set out to solve one of the solar system’s most puzzling mysteries when he went for a
run up a hill in Nice, France. Dr. Batygin, a Caltech researcher, best known for his contributions to the search
for the solar system’s missing “Planet Nine,” spotted a beer bottle. At a steep, 20 degree grade, he wondered why
it wasn’t rolling down the hill. He realized there was a breeze at his back holding the bottle in place. Then he
had a thought that would only pop into the mind of a theoretical astrophysicist: “Oh! This is how Europa formed.”
Europa is one of Jupiter’s four large Galilean moons. And in a paper published Monday in the Astrophysical Journal,
Dr. Batygin and a co-author, Alessandro Morbidelli, a planetary scientist at the Côte d’Azur Observatory in France,
present a theory explaining how some moons form around gas giants like Jupiter and Saturn, suggesting that
millimeter-sized grains of hail produced during the solar system’s formation became trapped around these massive
worlds, taking shape one at a time into the potentially habitable moons we know today.
"""

# Make sure our texts can be used in an SQL statement.
MY_TEXT = MY_TEXT.replace("'", "''")

# Classes, not seen during model training.
MY_LABELS = 'space & cosmos, scientific discovery, microbiology, robots, archeology'

Let's run the zero shot text classification model. We will save the result in the variable `udf_output` to support automatic testing of this notebook.

The `TE_ZERO_SHOT_TEXT_CLASSIFICATION_UDF` takes various input parameters:

* device_id: To run the UDF on a GPU, specify the valid cuda device ID.
* bucketfs_conn: The BucketFS connection name.
* sub_dir: The directory where the model is stored in the BucketFS.
* model_name: The name of the model to use for prediction.
* text_data: The text to be classified.
* candidate_labels: Labels, should be comma-separated.
* return_ranks: Either "ALL" or "HIGHEST".

You need to supply these parameters in the correct order. Further information can be found in the  <a href="https://github.com/exasol/transformers-extension/blob/main/doc/user_guide/user_guide.md" target="_blank" rel="noopener">User Guide</a>.

In [ ]:
%%sql --save udf_output
WITH MODEL_OUTPUT AS
(
    SELECT TE_ZERO_SHOT_TEXT_CLASSIFICATION_UDF(
        NULL,
        '{{ai_lab_config.bfs_connection_name}}',
        '{{ai_lab_config.bfs_model_subdir}}',
        '{{MODEL_NAME}}',
        '{{MY_TEXT}}',
        '{{MY_LABELS}}',
        'HIGHEST'
    )
)
SELECT label, score, error_message FROM MODEL_OUTPUT ORDER BY SCORE DESC

In this instance, we let the model return only the highest ranking result by setting `return_ranks = "HIGHEST"`. If we want to get all available results instead, we can run it with `return_ranks = "ALL"` instead:

In [ ]:
%%sql --save udf_output
WITH MODEL_OUTPUT AS
(
    SELECT TE_ZERO_SHOT_TEXT_CLASSIFICATION_UDF(
        NULL,
        '{{ai_lab_config.bfs_connection_name}}',
        '{{ai_lab_config.bfs_model_subdir}}',
        '{{MODEL_NAME}}',
        '{{MY_TEXT}}',
        '{{MY_LABELS}}',
        'ALL'
    )
)
SELECT label, score, error_message FROM MODEL_OUTPUT ORDER BY SCORE DESC

We select only some of the udf's output columns in these examples.  If you need more details, you can find information on all available output columns in the <a href="https://github.com/exasol/transformers-extension/blob/main/doc/user_guide/user_guide.md" target="_blank" rel="noopener">User Guide</a>.

The output of the model is sorted into the following columns by the udf:

* label: the label the model assigned for the input
* score: the confidence, with which the label was assigned
* rank: the rank of the label. In this context, all predictions/labels for one input are ranked by their score. rank=1 means best result/highest score.
* error_message: error occurring while executing the udf will be saved here

The code above shows how the model works on a toy example. However, the main purpose of having a model deployed in the database is to get a quick response for a batch input. The performance gain comes from two factors - localization and parallelization. The first means that the input data never crosses the machine boundaries. The second means that multiple instances of the model are processing the data on all available nodes in parallel.

Another advantage of making predictions within the database is the enhanced data security. The task of safeguarding privacy can be simplified giving the fact that the source data never leaves the database machine.

In a more practical application the text to be classified would be stored in a column of a database table. For example, if we wanted to get a label with the highest score for each row of the input table `MY_TEXT_TABLE`, where the text in question is in the column `MY_TEXT_COLUMN`, the SQL would look similar to this:
```
WITH MODEL_OUTPUT AS
(
    SELECT TE_ZERO_SHOT_TEXT_CLASSIFICATION_UDF(..., MY_TEXT_COLUMN, <MY_LABELS>) FROM MY_TEXT_TABLE
)
SELECT test_data, label FROM MODEL_OUTPUT WHERE rank=1;
```
Please note, that the response time observed on the provided example with a single input will not be scaled up linearly in case of multiple inputs. Much of the latency falls on loading the model into the CPU memory from BucketFS. This needs to be done only once regardless of the number of inputs.